# Summarizer for Financial Products and Concepts

This tool summarizes a given article about a financial product or concept, and provides links to 

In [1]:
import os
import json
from typing import List
from dotenv import load_dotenv

import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [2]:
load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")
if len(api_key) > 10 and api_key.startswith('sk-proj-'):
    print("Key looks good")
else:
    print("Something's wrong with the openai key")

MODEL = "gpt-4o-mini"
openai = OpenAI()

Key looks good


In [3]:
class Website:
    def __init__(self, url: str):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
        }
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]

    def get_contents(self, max_length=None):
        entire_content = f"Website title:\n{self.title}\nWebsite content:\n{self.text}\n\n"
        if max_length is None:
            return entire_content
        else: 
            if len(entire_content) > max_length:
                return entire_content[:max_length]
            return entire_content

In [4]:
find_relevant_links_prompt = """
    You are assisting in an application that builds a brochure about a financial product for potential clients.
    You are given a list of links extracted from a website. Some of these links provide background information about financial concepts, while others describe similar products. Your task is to:

    1. Filter out relevant links. Ignore links unrelated to finance.
    2. Categorize each relevant link into one of two groups:
        - "background" → Links explaining financial concepts relevant to the product (e.g., "What is a bond?").
        - "similar products" → Links about products that are alternatives or variations of the given financial product (e.g., "Private Credit" for a loan product).
    3. Return the result in valid JSON format as follows:

    {
        "links": [
            {"type": "similar products", "description": "Private credit", "url": "https://this-url.com/credit"},
            {"type": "background", "description": "What is a stock", "url": "https://this-url.com/stocks"}
        ]
    }

    - Ensure all URLs are properly formatted (no duplicates).
    - Do not include irrelevant links (e.g., company news, careers, or general homepage links).
"""

def get_find_links_user_prompt_for_website(website: Website) -> str:
    prompt = f"Here is the list of links of the website {website.url}. \n\n {website.links}.\n\n"
    prompt += "Please select the links relevant for building the financial product brochure. "
    prompt += "Exclude links such as terms of service, email, legal, and help page."
    return prompt

In [5]:
def get_relevant_links(website: Website):
    messages = [
        {"role": "system", "content": find_relevant_links_prompt},
        {"role": "user", "content": get_find_links_user_prompt_for_website(website)}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        response_format={"type": "json_object"}
    )
    content = response.choices[0].message.content
    return json.loads(content)

In [6]:
brochure_system_prompt = """
    You are a helpful and ethical assistant. You are provided with the product name, an explanation text scraped from a website, and a list of relevant links.

    The links include:
    - 'type' tags indicating whether they provide **background information** or refer to **similar products**.
    - 'description' tags explaining their relevance.
    - The actual URL.

    Your task is to write a **clear, neutral, and informative brochure** explaining the financial product.

    ### **Instructions:**
    1. **Target Audience:**
       - Assume that the readers **have limited financial knowledge**.
       - Use **simple, clear language** and avoid jargon unless explained.

    2. **Content & Structure:**
       - **Title:** Use `#` for the product name as the title.
       - **Introduction:** Start with a short, engaging summary of the product.
       - **Main Explanation:** Clearly explain the product’s purpose, how it works, and its benefits/risks.
       - **Numerical Example (if applicable):**
         - Include a practical example with numbers.
         - Make sure the math is properly formatted for markdown
       - **References Section:**
         - **Background links** (for understanding key concepts).
         - **Similar product links** (for exploring alternatives).

    3. **Citations & Ethical Considerations:**
       - At the **beginning** of the brochure, include a credit line:
         *"Information sourced from [Website Name] ([URL])"*.
       - Do **not** invent facts or speculate—use only the provided website content.

    4. **Response Format:**
       - Use **Markdown** (`#`, `##`, `- [Link](URL)`, `$math$`).
       - Ensure **proper sectioning** and **clean formatting**.
"""

def get_user_prompt_for_website(website: Website):
    prompt = f"""
        Please write a brochure for the financal product from this website: {website.url}.
        Product name: {website.title}.

        The text from the website: 
        {website.text}

        The list of links:
        {get_relevant_links(website)}
    """
    return prompt

In [7]:
def write_brochure_for_url(url: str):
    website = Website(url)
    model = OpenAI()
    user_prompt = get_user_prompt_for_website(website)
    messages = [
        {"role": "system", "content": brochure_system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    stream = model.chat.completions.create(
        model=MODEL,
        stream=True,
        messages=messages
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [8]:
write_brochure_for_url("https://www.investopedia.com/terms/e/etf.asp")

# Exchange-Traded Fund (ETF): How to Invest and What It Is

*Information sourced from Investopedia ([https://www.investopedia.com/terms/e/etf.asp](https://www.investopedia.com/terms/e/etf.asp))*

## Introduction
An Exchange-Traded Fund (ETF) is an investment option that combines features of stocks and mutual funds. It allows investors to buy a collection of various securities—such as stocks and bonds—through a single fund. ETFs are traded on major stock exchanges, meaning you can buy and sell them just like individual stocks.

## What is an ETF?
An ETF is a type of investment fund that holds multiple underlying assets, which allows investors to gain exposure to a wide range of securities. The purpose of an ETF is to track the performance of a specific index, commodity, or sector. For instance, the SPDR S&P 500 ETF (SPY) is designed to track the S&P 500 Index, which represents 500 of the largest U.S. companies.

### How Do ETFs Work?
ETFs are bought and sold on stock exchanges, and their prices fluctuate throughout the trading day. This is different from mutual funds, which only trade at the end of the market day. To invest in an ETF, you need a brokerage account where you can execute trades like you do with stocks.

### Benefits of ETFs:
- **Diversification**: An ETF consists of many assets, reducing the risk associated with investing in a single stock.
- **Lower Costs**: Most ETFs have lower expense ratios and fees compared to actively managed mutual funds.
- **Flexibility**: You can buy and sell ETFs like stocks during market hours, allowing for more trading flexibility.

### Risks of ETFs:
- **Market Volatility**: While diversified, an ETF's value can still fluctuate with the market.
- **Liquidity Issues**: Some ETFs may have low trading volumes, making them harder to buy or sell without affecting the price.
- **Complexity for New Investors**: Some specialized ETFs may require deeper knowledge of the underlying assets or strategies.

## Numerical Example
Let’s say you decide to invest in an ETF that holds 10 different companies. If each company is worth $10, the total value of the ETF would be:

\[
\text{Total Value} = 10 \text{ companies} \times 10 \text{ dollars each} = 100 \text{ dollars}
\]

If the price of the ETF increases to $12 per share, and you own 5 shares, the total value of your investment would be:

\[
\text{Total Investment Value} = 5 \text{ shares} \times 12 \text{ dollars per share} = 60 \text{ dollars}
\]

This illustrates the potential for both growth and risk in your investment.

## References
For more detailed information and related topics, consider exploring the following links:

### Background Information
- [What is an Exchange Traded Fund (ETF)?](https://www.investopedia.com/terms/e/etf.asp)
- [How ETFs Work](https://www.investopedia.com/articles/exchangetradedfunds/12/brief-history-exchange-traded-funds.asp)
- [Pros and Cons of ETFs](https://www.investopedia.com/articles/exchangetradedfunds/11/advantages-disadvantages-etfs.asp)

### Similar Products
- [ETFs vs. Mutual Funds](https://www.investopedia.com/ask/answers/09/etfs-vs-mutual-funds.asp)
- [ETF Dollar-Cost Averaging](https://www.investopedia.com/articles/mutualfund/05/etfdollarcost.asp)
- [Leveraged ETFs](https://www.investopedia.com/terms/l/leveraged-etf.asp)
- [Difference Between an ETF and an ETN](https://www.investopedia.com/terms/e/etn.asp)
- [Exchange-Traded Products](https://www.investopedia.com/terms/e/exchange-traded-products-etp.asp)
- [Best ETFs for Investing](https://www.investopedia.com/the-best-etfs-8781418) 

Exploring these resources can provide additional insights into ETFs and how they fit into your investment strategy.

In [9]:
write_brochure_for_url("https://www.investopedia.com/terms/p/putoption.asp")

# Put Option: What It Is, How It Works, and How to Trade Them

*"Information sourced from Investopedia ([https://www.investopedia.com/terms/p/putoption.asp](https://www.investopedia.com/terms/p/putoption.asp))*"

## Introduction
A put option is a financial contract that gives you the right—but not the obligation—to sell a specified amount of an underlying asset at a predetermined price, known as the strike price, within a set time frame. This financial tool can be crucial for managing risk or for speculating on price movements. 

## Main Explanation
### What is a Put Option?
In simple terms, a put option allows investors to protect themselves from falling prices of an asset (like stocks) or to profit from those price declines. If an investor believes that a stock's price will drop, they can buy a put option to sell that stock at a fixed price even if the market price plummets.

### How Does It Work?
When you buy a put option, you pay a premium (the cost of the option) for the right to sell the asset. If the asset's price falls below the strike price, the put option increases in value. If prices rise, the option may expire worthless, and the loss will be limited to the premium paid.

#### Key Concepts:
- **Strike Price:** The price at which you can sell the asset using the put option.
- **Expiration Date:** The date by which you must exercise the option.
- **Option Premium:** The price you pay to buy the option.
- **Intrinsic Value:** The difference between the strike price and the current market price of the asset.

### Benefits of Put Options:
- They provide a way to hedge against losses in an investment portfolio.
- They allow investors to profit from the decline in an asset's price without actually owning the asset.

### Risks of Put Options:
- If the market price of the asset does not fall below the strike price, you may lose the entire premium paid.
- These contracts can expire worthless if not exercised or sold before expiration.

## Numerical Example
Let’s say you buy one put option on a stock priced at $100 with a strike price of $90. You pay a premium of $2 per share.

1. If the stock price drops to $80, your put option is "in the money."
   - Intrinsic Value = Strike Price - Current Price = $90 - $80 = $10.
   - If you sell the option, it would likely be worth more than the premium paid.
  
2. If the stock price is above $90 when the option expires, the option may expire worthless. 
   - Your maximum loss in this case: 
   $2 (premium paid) * 100 shares = **$200.**

## References
### Background Links
- [What is a put option](https://www.investopedia.com/terms/p/putoption.asp)
- [Options and derivatives trading](https://www.investopedia.com/options-and-derivatives-trading-4689663)
- [Basics of options profitability](https://www.investopedia.com/articles/active-trading/091714/basics-options-profitability.asp)
- [Buying options](https://www.investopedia.com/articles/optioninvestor/09/buying-options.asp)
- [Call options](https://www.investopedia.com/terms/c/calloption.asp)
- [Strike price](https://www.investopedia.com/terms/s/strikeprice.asp)
- [Expiration date](https://www.investopedia.com/terms/e/expirationdate.asp)
- [Option premium](https://www.investopedia.com/terms/o/option-premium.asp)
- [Intrinsic value](https://www.investopedia.com/terms/i/intrinsicvalue.asp)
- [Out of the money](https://www.investopedia.com/terms/o/outofthemoney.asp)
- [In the money](https://www.investopedia.com/terms/i/inthemoney.asp)
- [Volatility skew](https://www.investopedia.com/terms/v/volatility-skew.asp)

### Similar Product Links
- [Covered call](https://www.investopedia.com/terms/c/coveredcall.asp)
- [Married put](https://www.investopedia.com/terms/m/marriedput.asp)
- [Bull call spread](https://www.investopedia.com/terms/b/bullcallspread.asp)
- [Iron condor](https://www.investopedia.com/terms/i/ironcondor.asp)
- [Straddle](https://www.investopedia.com/terms/s/straddle.asp)
- [Strangle](https://www.investopedia.com/terms/s/strangle.asp)
- [Butterfly spread](https://www.investopedia.com/terms/b/butterflyspread.asp)
- [Credit spread](https://www.investopedia.com/ask/answers/042215/whats-difference-between-credit-spread-and-debt-spread.asp)

This brochure provides a simple overview of put options, explaining their usage, benefits, risks, and what you need to consider when trading them. Whether you use them for hedging against stock price drops or speculation, understanding put options is important for any investor.

In [10]:
write_brochure_for_url("https://www.investopedia.com/terms/s/shortselling.asp")

# Short Selling: Your Step-by-Step Guide for Shorting Stocks

*Information sourced from Investopedia ([https://www.investopedia.com/terms/s/shortselling.asp](https://www.investopedia.com/terms/s/shortselling.asp))*

## Introduction

Short selling is an investment strategy that allows traders to profit from a decline in the price of a stock. Though it may seem counterintuitive to bet against a company's success, many investors employ short selling to capitalize on anticipated downturns or to hedge against potential losses in their portfolios.

## What is Short Selling?

Short selling involves borrowing shares of a stock and selling them on the market with the intention of buying them back later at a lower price. Here’s how it works:

1. **Borrow:** An investor borrows shares from a broker or another investor and sells them.
2. **Wait:** The investor hopes the stock price will drop.
3. **Buy Back:** Once the price falls, the investor buys back the shares at the lower price.
4. **Return:** The shares are returned to the lender, and the difference in prices (minus fees) is profit.

This strategy can be risky, as the price of a stock can rise instead of fall, leading to potential losses.

## How Short Selling Works

To engage in short selling, you typically need a margin account, where the broker allows you to borrow money to buy stocks. Here are the steps involved:

1. **Open a Margin Account:** Establish this account with a broker to start borrowing shares.
2. **Identify a Stock:** Determine which stock you want to short, usually based on research or market trends.
3. **Locate Borrowable Shares:** Your broker helps find shares available for shorting.
4. **Place the Order:** Execute the short sale through your brokerage.
5. **Monitor the Investment:** Keep an eye on the stock price.
6. **Close the Short Position:** Once the price drops, buy back the shares to close your position.

### Example Calculation

Imagine you short-sell 100 shares of XYZ company, currently priced at $50 per share:
- **Initial Sale:** 100 shares sold at $50 = **$5,000**
- If the price drops to $40:
  - **Buy Back Cost:** 100 shares bought at $40 = **$4,000**
  - **Profit:** $5,000 (initial sale) - $4,000 (buy back) = **$1,000 profit**

Conversely, if the price rises to $65 and you decide to close your position:
- **Loss Calculation:**
  - **Cost to Buy Back:** 100 shares at $65 = **$6,500**
  - **Loss:** $5,000 (initial sale) - $6,500 (buy back) = **$1,500 loss**

## Pros and Cons of Short Selling

### Advantages
- The potential for high profits.
- Minimal initial capital required.
- Can be used as a hedge against other investments.

### Disadvantages
- Potentially unlimited losses if the stock price increases.
- Incur costs related to margin interest.
- Requires careful timing and expert knowledge.

## Regulations

Short selling is regulated in various markets to prevent manipulation. In the U.S., traders must comply with the Securities and Exchange Commission (SEC) rules, which include ensuring the shares being sold short are borrowed.

## Conclusion

Short selling can be a powerful investment strategy when used correctly, allowing traders to benefit from market declines. However, it carries significant risks and requires a solid understanding of the markets.

## References

### Background Information
- [What is short selling?](https://www.investopedia.com/terms/s/shortselling.asp)
- [How shorting a stock works](https://www.investopedia.com/articles/trading/10/top-ten-rules-for-trading.asp)
- [Short selling costs](https://www.investopedia.com/articles/trading/06/daytradingretail.asp)
- [Short selling strategies](https://www.investopedia.com/articles/trading/09/risk-management.asp)
- [Advantages and disadvantages of short selling](https://www.investopedia.com/articles/stocks/07/why-short-selling-legal-brief-history.asp)
- [Regulations of short selling](https://www.sec.gov/investor/pubs/regsho.htm)
- [Example of short selling](https://www.investopedia.com/articles/optioninvestor/03/073003.asp)
- [Why do short sellers have to borrow shares?](https://www.investopedia.com/ask/answers/011315/how-it-possible-trade-stock-you-dont-own-done-short-selling.asp)

### Similar Products
- [Difference between short selling and put options](https://www.investopedia.com/articles/trading/092613/difference-between-short-selling-and-put-options.asp) 

By understanding how short selling works and following the guidelines, investors can effectively navigate this complex trading strategy. Be sure to do your research and consult financial advisors when needed!